# load the data from excel

In [4]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [6]:
# read the excel data
df = pd.read_csv('/app/notebooks/rapprochements/RPLS/export.csv', sep=";",
                 dtype={})

# print the df length
print('The length of the dataframe is: ', len(df))

The length of the dataframe is:  5161392


In [7]:
# display df columns
print(df.columns)

Index(['ident_rpls', 'adresse_brut', 'array_agg'], dtype='object')


# map to inputs

In [13]:
filtered_df = df[df["adresse_brut"].notna()]

In [14]:
def row_to_input(df_row):
    return {
        "ext_id" : df_row["ident_rpls"],
        "address": df_row["adresse_brut"]
    }

inputs = list(filtered_df.apply(row_to_input, axis=1))

In [15]:
import random
#inputs = random.sample(inputs, 1000)
print(len(inputs))
print(inputs[0])

4192661
{'ext_id': 22613394, 'address': '10 Impasse Du Poitou 57150 Creutzwald'}


In [ ]:
from batid.services.guess_bdg_new import Guesser, GeocodeNameHandler, GeocodeAddressHandler, ClosestFromPointHandler
guesses_path = "./guesses.json"
guesser = Guesser(batch_size=4000)
guesser.handlers = [
    GeocodeAddressHandler()
]
guesser.create_work_file(list(inputs), guesses_path)
guesser.guess_work_file(guesses_path)

  0%|          | 0/1049 [00:00<?, ?it/s]

In [10]:
guesser = Guesser()
guesser.load_work_file(guesses_path)
guesser.report()

-- Report --
Number of rows: 1000
Number of match: 764 (76.40%)

-- finished_steps --
Rows with finished_steps closest_from_point: 0 (0.00%)
Rows with finished_steps geocode_address: 1000 (100.00%)
Rows with finished_steps geocode_name: 0 (0.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
precise_address_match    764
Name: count, dtype: int64

-- match_reasons : % --
match_reason
precise_address_match    76.4
Name: count, dtype: float64

-- Inputs --
rows with ban_id: 801 (80.10%)


In [11]:
guesser.to_csv("/app/notebooks/rapprochements/RPLS/RPLS_out.csv", ext_id_col_name="ext_id")

In [12]:
os.getcwd()

'/app/notebooks/rapprochements/Ramses'